# CS 7641 HW3 Code

I'd like to provide proper attribution of code use to Kyle West. His original code can be found at the following link.
https://github.com/kylewest520/CS-7641---Machine-Learning/blob/master/Assignment%203%20Unsupervised%20Learning/CS%207641%20HW3%20Code.ipynb



This file will provide analysis for 2 Different clustering algorithms and 4 dimensionality reduction algorithms on two different datasets.

Datasets: Wine Quality, Credit Card Defaults.

Clustering Algorithms: K-Means, Expectation Maximization

Dimensionality Reduction: Principal Components Analysis, Independent Components Analysis, Random Components Analysis, Random Forest Classifier Components Analysis

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate, train_test_split, GridSearchCV, learning_curve
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import itertools
import timeit
import os
%matplotlib inline

In [ ]:
#---- change to your directory where data files are located ---#
#os.chdir(r"C:\...") #change this to your current working directory

In [ ]:
#Read data in and replace last column, drop ID column
df_default = pd.read_csv("defaultCreditCards.csv")
df_default.columns = ['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'y']
df_default.drop('ID', axis=1, inplace=True)

In [ ]:
df_default.head()

In [ ]:
#Function to scale numeric features
def scaleDefaultData(df, scale=False, encode = False):    
    if scale == True:
        numCols = ['LIMIT_BAL', 'AGE', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']
        df_num = df[numCols]
        df_stand = (df_num-df_num.min()) / (df_num.max()-df_num.min())
        df_cat = df.drop(numCols, axis=1)  
        df = pd.concat([df_stand, df_cat], axis=1)
        if encode == True:
            col_1hot =  ['SEX', 'EDUCATION', 'MARRIAGE', 'PAY_0',
                           'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']
            df_1hot = df[col_1hot].astype('category')
            df_1hot = pd.get_dummies(df_1hot).astype('int')
            df_others = df.drop(col_1hot, axis=1)
            df = pd.concat([df_1hot, df_others], axis=1)
    return df

df_default_scaled = scaleDefaultData(df_default, scale=True, encode=True)

In [ ]:
#Load in wine quality dataset
df_wine = pd.read_csv('wineQuality.txt', sep=';')

#If wine rating > 5, set quality = 1, else 0.
df_wine['quality'] = df_wine['quality'].apply(lambda x: 1 if x > 5 else 0)

In [ ]:
#Function to scale wine numeric features
def scaleWineData(df):
    numCols = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol']
    df_num = df[numCols]
    df_stand = (df_num-df_num.min()) / (df_num.max()-df_num.min())
    df_cat = df.drop(numCols, axis=1)  
    df = pd.concat([df_stand, df_cat], axis=1)
    return df

df_wine_scaled = scaleWineData(df_wine)

In [ ]:
#Split both datasets into x and y.
def splitData(df_default, df_wine):
    X1 = np.array(df_default.values[:, 0:-1])
    y1 = np.array(df_default.values[:, -1])
    X2 = np.array(df_wine.values[:, 0:-1])
    y2 = np.array(df_wine.values[:, -1])
    
    return X1, y1, X2, y2

#Train test split
def trainTestSplit(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=100)
    return X_train, X_test, y_train, y_test
    

In [ ]:
#Split both scaled datasets into X and Y.
X_default, y_default, X_wine, y_wine = splitData(df_wine_scaled, df_wine_scaled)

#---Create train and test sets for both datasets---#

#Default
X_default_train, X_default_test, y_default_train, y_default_test = trainTestSplit(X_default, y_default)

#Wine
X_wine_train, X_wine_test, y_wine_train, y_wine_test = trainTestSplit(X_wine, y_wine)

In [ ]:
#--Helper functions --##
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_validate, train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import itertools
import timeit
from collections import Counter
from sklearn.metrics.pairwise import pairwise_distances

def plot_learning_curve(clf, X, y, title="Insert Title"):
    
    n = len(y)
    train_mean = []; train_std = [] #model performance score (f1)
    cv_mean = []; cv_std = [] #model performance score (f1)
    fit_mean = []; fit_std = [] #model fit/training time
    pred_mean = []; pred_std = [] #model test/prediction times
    train_sizes=(np.linspace(.05, 1.0, 20)*n).astype('int')  
    
    for i in train_sizes:
        #print(i)
        idx = np.random.randint(X.shape[0], size=i)
        X_subset = X[idx,:]
        y_subset = y[idx]
        scores = cross_validate(clf, X_subset, y_subset, cv=10, scoring='f1', n_jobs=-1, return_train_score=True)
        
        train_mean.append(np.mean(scores['train_score'])); train_std.append(np.std(scores['train_score']))
        cv_mean.append(np.mean(scores['test_score'])); cv_std.append(np.std(scores['test_score']))
        fit_mean.append(np.mean(scores['fit_time'])); fit_std.append(np.std(scores['fit_time']))
        pred_mean.append(np.mean(scores['score_time'])); pred_std.append(np.std(scores['score_time']))
    
    train_mean = np.array(train_mean); train_std = np.array(train_std)
    cv_mean = np.array(cv_mean); cv_std = np.array(cv_std)
    fit_mean = np.array(fit_mean); fit_std = np.array(fit_std)
    pred_mean = np.array(pred_mean); pred_std = np.array(pred_std)
    
    plot_LC(train_sizes, train_mean, train_std, cv_mean, cv_std, title)
    plot_times(train_sizes, fit_mean, fit_std, pred_mean, pred_std, title)
    
    return train_sizes, train_mean, fit_mean, pred_mean
    

def plot_LC(train_sizes, train_mean, train_std, cv_mean, cv_std, title):
    
    plt.figure()
    plt.title("Learning Curve: "+ title)
    plt.xlabel("Training Examples")
    plt.ylabel("Model F1 Score")
    plt.fill_between(train_sizes, train_mean - 2*train_std, train_mean + 2*train_std, alpha=0.1, color="b")
    plt.fill_between(train_sizes, cv_mean - 2*cv_std, cv_mean + 2*cv_std, alpha=0.1, color="r")
    plt.plot(train_sizes, train_mean, 'o-', color="b", label="Training Score")
    plt.plot(train_sizes, cv_mean, 'o-', color="r", label="Cross-Validation Score")
    plt.legend(loc="best")
    plt.show()
    
    
def plot_times(train_sizes, fit_mean, fit_std, pred_mean, pred_std, title):
    
    plt.figure()
    plt.title("Modeling Time: "+ title)
    plt.xlabel("Training Examples")
    plt.ylabel("Training Time (s)")
    plt.fill_between(train_sizes, fit_mean - 2*fit_std, fit_mean + 2*fit_std, alpha=0.1, color="b")
    plt.fill_between(train_sizes, pred_mean - 2*pred_std, pred_mean + 2*pred_std, alpha=0.1, color="r")
    plt.plot(train_sizes, fit_mean, 'o-', color="b", label="Training Time (s)")
    plt.plot(train_sizes, pred_std, 'o-', color="r", label="Prediction Time (s)")
    plt.legend(loc="best")
    plt.show()
    
    
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion Matrix', cmap=plt.cm.Blues):

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(2), range(2)):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    
    
def final_classifier_evaluation(clf,X_train, X_test, y_train, y_test):
    
    start_time = timeit.default_timer()
    clf.fit(X_train, y_train)
    end_time = timeit.default_timer()
    training_time = end_time - start_time
    
    start_time = timeit.default_timer()    
    y_pred = clf.predict(X_test)
    end_time = timeit.default_timer()
    pred_time = end_time - start_time
    
    auc = roc_auc_score(y_test, y_pred)
    f1 = f1_score(y_test,y_pred)
    accuracy = accuracy_score(y_test,y_pred)
    precision = precision_score(y_test,y_pred)
    recall = recall_score(y_test,y_pred)
    cm = confusion_matrix(y_test,y_pred)

    print("Model Evaluation Metrics Using Untouched Test Dataset")
    print("*****************************************************")
    print("Model Training Time (s):   "+"{:.5f}".format(training_time))
    print("Model Prediction Time (s): "+"{:.5f}\n".format(pred_time))
    print("F1 Score:  "+"{:.2f}".format(f1))
    print("Accuracy:  "+"{:.2f}".format(accuracy)+"     AUC:       "+"{:.2f}".format(auc))
    print("Precision: "+"{:.2f}".format(precision)+"     Recall:    "+"{:.2f}".format(recall))
    print("*****************************************************")
    plt.figure()
    plot_confusion_matrix(cm, classes=["0","1"], title='Confusion Matrix')
    plt.show()
    
def cluster_predictions(Y,clusterLabels):
    assert (Y.shape == clusterLabels.shape)
    pred = np.empty_like(Y)
    for label in set(clusterLabels):
        #print(label)
        mask = clusterLabels == label
        sub = Y[mask]
        target = Counter(sub).most_common(1)[0][0]
        pred[mask] = target
#    assert max(pred) == max(Y)
#    assert min(pred) == min(Y)    
    return pred

def pairwiseDistCorr(X1,X2):
    assert X1.shape[0] == X2.shape[0]
    
    d1 = pairwise_distances(X1)
    d2 = pairwise_distances(X2)
    return np.corrcoef(d1.ravel(),d2.ravel())[0,1]

def plot_cluster_scatter(X, y, cluster_labels):
    plt.scatter(range(len(X)), clusters, c=y)

def plot_cluster(km, X_data, y_data, clusters, title="First and Second Dimension Clusters"):
    x = []
    y = []
    for center in km.cluster_centers_:
        x.append(center[0])
        y.append(center[1])
    plt.figure(figsize=(5,5))
    plt.scatter(X_data[:, 0], X_data[:,1], c=clusters, cmap='rainbow')
    plt.scatter(x, y, marker='*', color='black', s=300, edgecolors='orange')
    plt.title(title)

def km_complexity(vals, n_clusters=20, n_init=10):
    ssd = []
    for i in vals:
        km = KMeans(n_clusters=n_clusters,n_init=i,random_state=100, n_jobs=-1)
        km.fit(X_wine)
        ssd.append(km.inertia_)
        #preds = km.predict(X_wine)
    
        #y_mode_vote = cluster_predictions(y_wine, km.labels_)
        #acc_score = accuracy_score(y_wine, y_mode_vote)
        #print("F1 score at " + str(n_clusters) + " clusters and " + str(i) + " random restarts is " + str(acc_score))
    
    plt.plot(vals, ssd)

Neural Net

In [ ]:
from sklearn.neural_network import MLPClassifier

def hyperNN(X_train, y_train, X_test, y_test, title):

    f1_test = []
    f1_train = []
    hlist = np.linspace(1,150,30).astype('int')
    for i in hlist:         
            clf = MLPClassifier(hidden_layer_sizes=(i,), solver='adam', activation='logistic', 
                                learning_rate_init=0.05, random_state=100)
            clf.fit(X_train, y_train)
            y_pred_test = clf.predict(X_test)
            y_pred_train = clf.predict(X_train)
            f1_test.append(f1_score(y_test, y_pred_test))
            f1_train.append(f1_score(y_train, y_pred_train))
      
    plt.plot(hlist, f1_test, 'o-', color='r', label='Test F1 Score')
    plt.plot(hlist, f1_train, 'o-', color = 'b', label='Train F1 Score')
    plt.ylabel('Model F1 Score')
    plt.xlabel('No. Hidden Units')
    
    plt.title(title)
    plt.legend(loc='best')
    plt.tight_layout()
    plt.show()
    
    
def NNGridSearchCV(X_train, y_train):
    #parameters to search:
    #number of hidden units
    #learning_rate
    h_units = [5, 10, 20, 30, 40, 50, 75, 100]
    learning_rates = [0.01, 0.05, .1]
    param_grid = {'hidden_layer_sizes': h_units, 'learning_rate_init': learning_rates}

    net = GridSearchCV(estimator = MLPClassifier(solver='adam',activation='logistic',random_state=100),
                       param_grid=param_grid, cv=10, n_jobs=-1)
    net.fit(X_train, y_train)
    print("Per Hyperparameter tuning, best parameters are:")
    print(net.best_params_)
    return net.best_params_['hidden_layer_sizes'], net.best_params_['learning_rate_init']

This section will implement k-means clustering for both datasets. Our objectives are to:
1. Determine the best number of clusters for each dataset by using the elbow inspection method on silhouette score.
2. Describe the attributes which make up each cluster.
3. Score each cluster with an accuracy since technically we do have labels available for these datasets (labels are not used when determining clusters).

Since k-Means is susceptible to get stuck in local optima due to the random selection of initial cluster centers, I will report the average metrics over 5 models for each number of k clusters.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score as sil_score, f1_score, homogeneity_score
import matplotlib.pyplot as plt

np.random.seed(0)

def run_kmeans(X,y,title):

    kclusters = list(np.arange(2,50,2))
    sil_scores = []; f1_scores = []; homo_scores = []; train_times = []; sum_squared_distances = []

    for k in kclusters:
        #print(k)
        start_time = timeit.default_timer()
        km = KMeans(n_clusters=k, n_init=10,random_state=100,n_jobs=-1).fit(X)
        end_time = timeit.default_timer()
        train_times.append(end_time - start_time)
        #sil_scores.append(sil_score(X, km.labels_))
        #y_mode_vote = cluster_predictions(y,km.labels_)
        #f1_scores.append(f1_score(y, y_mode_vote))
        #homo_scores.append(homogeneity_score(y, km.labels_))
        sum_squared_distances.append(km.inertia_)

    # plot model training time
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(kclusters, train_times)
    plt.grid(True)
    plt.xlabel('No. Clusters')
    plt.ylabel('Training Time (s)')
    plt.title('KMeans Training Time: '+ title)
    plt.show()
    
    #Plot squared distances
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(kclusters, sum_squared_distances, 'bx-')
    plt.grid(True)
    plt.xlabel('No. Clusters')
    plt.ylabel('Squared Distance')
    plt.title('Squared Distances from cluster centers')
    plt.show()
    
    return sum_squared_distances
    
def evaluate_kmeans(km, X, y):
    start_time = timeit.default_timer()
    km.fit(X, y)
    end_time = timeit.default_timer()
    training_time = end_time - start_time
    
    y_mode_vote = cluster_predictions(y,km.labels_)
    #print(y_mode_vote)
    auc = roc_auc_score(y, y_mode_vote)
    f1 = f1_score(y, y_mode_vote)
    accuracy = accuracy_score(y, y_mode_vote)
    precision = precision_score(y, y_mode_vote)
    recall = recall_score(y, y_mode_vote)
    cm = confusion_matrix(y, y_mode_vote)

    print("Model Evaluation Metrics Using Mode Cluster Vote")
    print("*****************************************************")
    print("Model Training Time (s):   "+"{:.2f}".format(training_time))
    print("No. Iterations to Converge: {}".format(km.n_iter_))
    print("F1 Score:  "+"{:.2f}".format(f1))
    print("Accuracy:  "+"{:.2f}".format(accuracy)+"     AUC:       "+"{:.2f}".format(auc))
    print("Precision: "+"{:.2f}".format(precision)+"     Recall:    "+"{:.2f}".format(recall))
    print("*****************************************************")
    plt.figure()
    plot_confusion_matrix(cm, classes=["0","1"], title='Confusion Matrix')
    plt.show()    

In [ ]:
wine_squared_distances = run_kmeans(X_wine, y_wine,'Wine Data')
km = KMeans(n_clusters=20,n_init=10,random_state=100,n_jobs=-1)
clusters=km.fit_predict(X_wine)
evaluate_kmeans(km,X_wine, y_wine)
plot_cluster_scatter(X_wine, y_wine, clusters)
plot_cluster(km, X_wine, y_wine, clusters)

In [ ]:
km_complexity([10,20,30,40,50,60,70,80], n_clusters=20)

In [ ]:
default_squared_distances = run_kmeans(X_default, y_default,'Default Data')
km = KMeans(n_clusters=25,n_init=10,random_state=100,n_jobs=-1)
clusters = km.fit_predict(X_default)
evaluate_kmeans(km,X_default,y_default)
plot_cluster_scatter(X_default, y_default, clusters)
#plot_cluster(km, X_default, y_default, clusters)
#km_complexity([10,20,30,40,50,60,70,80])


In [ ]:
from sklearn.mixture import GaussianMixture as EM
from sklearn.metrics import silhouette_score as sil_score, f1_score, homogeneity_score
import matplotlib.pyplot as plt

np.random.seed(0)

def run_EM(X,y,title):

    #kdist =  [2,3,4,5]
    #kdist = list(range(2,51))
    kdist = list(np.arange(2,100,5))
    sil_scores = []; f1_scores = []; homo_scores = []; train_times = []; aic_scores = []; bic_scores = []; log_lik = []
    
    for k in kdist:
        start_time = timeit.default_timer()
        em = EM(n_components=k,covariance_type='diag',n_init=1,warm_start=True,random_state=100).fit(X)
        end_time = timeit.default_timer()
        train_times.append(end_time - start_time)
        
        labels = em.predict(X)
        #sil_scores.append(sil_score(X, labels))
        #y_mode_vote = cluster_predictions(y,labels)
        #f1_scores.append(f1_score(y, y_mode_vote))
        #homo_scores.append(homogeneity_score(y, labels))
        aic_scores.append(em.aic(X))
        bic_scores.append(em.bic(X))
        log_lik.append(em.lower_bound_)
        
    # elbow curve for silhouette score
    #fig = plt.figure()
    #ax = fig.add_subplot(111)
    #ax.plot(kdist, sil_scores)
    #plt.grid(True)
    #plt.xlabel('No. Distributions')
    #plt.ylabel('Avg Silhouette Score')
    #plt.title('Elbow Plot for EM: '+ title)
    #plt.show()
   
    # plot homogeneity scores
    #fig = plt.figure()
    #ax = fig.add_subplot(111)
    #ax.plot(kdist, homo_scores)
    #plt.grid(True)
    #plt.xlabel('No. Distributions')
    #plt.ylabel('Homogeneity Score')
    #plt.title('Homogeneity Scores EM: '+ title)
    #plt.show()

    # plot f1 scores
    #fig = plt.figure()
    #ax = fig.add_subplot(111)
    #ax.plot(kdist, f1_scores)
    #plt.grid(True)
    #plt.xlabel('No. Distributions')
    #plt.ylabel('F1 Score')
    #plt.title('F1 Scores EM: '+ title)
    #plt.show()

    # plot model AIC and BIC
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(kdist, aic_scores, label='AIC')
    ax.plot(kdist, bic_scores,label='BIC')
    plt.grid(True)
    plt.xlabel('No. Distributions')
    plt.ylabel('Model Complexity Score')
    plt.title('EM Model Complexity: '+ title)
    plt.legend(loc="best")
    plt.show()
    
    # plot model training time
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(kdist, train_times)
    plt.grid(True)
    plt.xlabel('No. Clusters')
    plt.ylabel('Training Time (s)')
    plt.title('EM Training Time: '+ title)
    plt.show()
    
    #Plot
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(kdist, log_lik)
    plt.grid(True)
    plt.xlabel('No. Clusters')
    plt.ylabel('Log likelihood')
    plt.title('EM Log likelihood '+ title)
    plt.show()
    
    return aic_scores, bic_scores, log_lik
    
def evaluate_EM(em, X, y):
    start_time = timeit.default_timer()
    em.fit(X, y)
    end_time = timeit.default_timer()
    training_time = end_time - start_time
    
    labels = em.predict(X)
    y_mode_vote = cluster_predictions(y,labels)
    auc = roc_auc_score(y, y_mode_vote)
    f1 = f1_score(y, y_mode_vote)
    accuracy = accuracy_score(y, y_mode_vote)
    precision = precision_score(y, y_mode_vote)
    recall = recall_score(y, y_mode_vote)
    cm = confusion_matrix(y, y_mode_vote)

    print("Model Evaluation Metrics Using Mode Cluster Vote")
    print("*****************************************************")
    print("Model Training Time (s):   "+"{:.2f}".format(training_time))
    print("No. Iterations to Converge: {}".format(em.n_iter_))
    print("Log-likelihood Lower Bound: {:.2f}".format(em.lower_bound_))
    print("F1 Score:  "+"{:.2f}".format(f1))
    print("Accuracy:  "+"{:.2f}".format(accuracy)+"     AUC:       "+"{:.2f}".format(auc))
    print("Precision: "+"{:.2f}".format(precision)+"     Recall:    "+"{:.2f}".format(recall))
    print("*****************************************************")
    plt.figure()
    plot_confusion_matrix(cm, classes=["0","1"], title='Confusion Matrix')
    plt.show()

In [ ]:
wine_aic, wine_bic, wine_log_lik = run_EM(X_wine, y_wine,'Wine Data')
em = EM(n_components=24,covariance_type='diag',n_init=1,warm_start=True,random_state=100).fit(X_wine)
preds = em.predict(X_wine)
evaluate_EM(em,X_wine, y_wine)
plt.scatter(range(len(X_wine)), preds, c=y_wine, cmap='plasma')
#df = pd.DataFrame(em.means_)
#print(df)
#df.to_csv("Phishing EM Component Means.csv")

In [ ]:
EM_improvement = {}
for covType in ['diag', 'spherical', 'full']:
    log_liks = []
    for n in range(1, 11):
        em = EM(n_components=24,covariance_type=covType,n_init=n,warm_start=True,random_state=100).fit(X_wine)
        log_liks.append(em.lower_bound_)
        #print("EM log-likelihood for " + covType + " covariance type and " + str(n) + " random restarts is " + str(em.lower_bound_))
    EM_improvement[covType] = log_liks



In [ ]:
plt.plot(list(range(1,11)), EM_improvement['diag'], label='diag', c='blue')
plt.plot(list(range(1,11)), EM_improvement['spherical'], label='spherical', c='red')
plt.plot(list(range(1,11)), EM_improvement['full'], label='full', c='green')
plt.ylabel("Log likelihood")
plt.xlabel("random restarts")
plt.legend(loc="best")

In [ ]:
default_aic, default_bic, default_log_lik = run_EM(X_default, y_default,'Default Data')
em = EM(n_components=20,covariance_type='diag',n_init=1,warm_start=True,random_state=100).fit(X_default)
preds = em.predict(X_default)
evaluate_EM(em,X_default, y_default)
plt.scatter(range(len(X_default)), preds, c=y_default, cmap='plasma')

This section will implement 4 different dimensionality reduction techniques on both the phishing and the banking dataset. Then, k-means and EM clustering will be performed for each (dataset * dim_reduction) combination to see how the clustering compares with using the full datasets. The 4 dimensionality reduction techniques are:
- Principal Components Analysis (PCA). Optimal number of PC chosen by inspecting % variance explained and the eigenvalues.
- Independent Components Analysis (ICA). Optimal number of IC chosen by inspecting kurtosis.
- Random Components Analysis (RCA) (otherwise known as Randomized Projections). Optimal number of RC chosen by inspecting reconstruction error.
- Random Forest Classifier (RFC). Optimal number of components chosen by feature importance.

In [ ]:
from sklearn.decomposition import PCA, FastICA as ICA
from sklearn.random_projection import GaussianRandomProjection as GRP, SparseRandomProjection as RCA
from sklearn.ensemble import RandomForestClassifier as RFC
from itertools import product
from collections import defaultdict

def run_PCA(X,y,title):
    
    pca = PCA(random_state=5).fit(X) #for all components
    cum_var = np.cumsum(pca.explained_variance_ratio_)

    fig, ax1 = plt.subplots()
    ax1.plot(list(range(len(pca.explained_variance_ratio_))), cum_var, 'b-')
    ax1.set_xlabel('Principal Components')
    # Make the y-axis label, ticks and tick labels match the line color.
    ax1.set_ylabel('Cumulative Explained Variance Ratio', color='b')
    ax1.tick_params('y', colors='b')
    plt.grid(False)

    ax2 = ax1.twinx()
    ax2.plot(list(range(len(pca.singular_values_))), pca.singular_values_, 'm-')
    ax2.set_ylabel('Eigenvalues', color='m')
    ax2.tick_params('y', colors='m')
    plt.grid(False)

    plt.title("PCA Explained Variance and Eigenvalues: "+ title)
    fig.tight_layout()
    plt.show()
    
def run_ICA(X,y,title):
    
    dims = list(np.arange(2,(X.shape[1]-1),3))
    #dims = list(np.arange(2,80,3))
    dims.append(X.shape[1])
    ica = ICA(random_state=5)
    kurt = []

    for dim in dims:
        print(dim)
        ica.set_params(n_components=dim)
        tmp = ica.fit_transform(X)
        tmp = pd.DataFrame(tmp)
        tmp = tmp.kurt(axis=0)
        kurt.append(tmp.abs().mean())

    plt.figure()
    plt.title("ICA Kurtosis: "+ title)
    plt.xlabel("Independent Components")
    plt.ylabel("Avg Kurtosis Across IC")
    plt.plot(dims, kurt, 'b-')
    plt.grid(False)
    plt.show()

def run_RCA(X,y,title):
    
    dims = list(np.arange(2,(X.shape[1]-1),3))
    #dims = list(np.arange(2,80,3))
    dims.append(X.shape[1])
    tmp = defaultdict(dict)

    for i,dim in product(range(5),dims):
        #print(i)
        rp = RCA(random_state=i, n_components=dim)
        tmp[dim][i] = pairwiseDistCorr(rp.fit_transform(X), X)
    print(tmp)
    tmp = pd.DataFrame(tmp).T
    print(tmp)
    mean_recon = tmp.mean(axis=1).tolist()
    std_recon = tmp.std(axis=1).tolist()


    fig, ax1 = plt.subplots()
    ax1.plot(dims,mean_recon, 'b-')
    ax1.set_xlabel('Random Components')
    # Make the y-axis label, ticks and tick labels match the line color.
    ax1.set_ylabel('Mean Reconstruction Correlation', color='b')
    ax1.tick_params('y', colors='b')
    plt.grid(False)

    ax2 = ax1.twinx()
    ax2.plot(dims,std_recon, 'm-')
    ax2.set_ylabel('STD Reconstruction Correlation', color='m')
    ax2.tick_params('y', colors='m')
    plt.grid(False)

    plt.title("Random Components for 5 Restarts: "+ title)
    fig.tight_layout()
    plt.show()
    
def run_RFC(X,y,df_original):
    rfc = RFC(n_estimators=500,min_samples_leaf=round(len(X)*.01),random_state=5,n_jobs=-1)
    imp = rfc.fit(X,y).feature_importances_ 
    imp = pd.DataFrame(imp,columns=['Feature Importance'],index=df_original.columns[:-1])
    imp.sort_values(by=['Feature Importance'],inplace=True,ascending=False)
    imp['Cum Sum'] = imp['Feature Importance'].cumsum()
    imp = imp[imp['Cum Sum']<=0.95]
    top_cols = imp.index.tolist()
    return imp, top_cols

In [ ]:
run_PCA(X_wine, y_wine,"Wine Data")
run_ICA(X_wine, y_wine,"Wine Data")
run_RCA(X_wine, y_wine,"Wine Data")
imp_wine, topcols_wine = run_RFC(X_wine, y_wine, df_wine_scaled)
plt.figure(figsize=(5,5))
fi = sns.barplot(x=imp_wine.index, y=imp_wine['Feature Importance'])
fi.set_xticklabels(fi.get_xticklabels(), rotation=90)

In [ ]:
run_PCA(X_default, y_default,"Default Data")
run_ICA(X_default, y_default,"Default Data")
#run_RCA(X_default, y_default,"Default Data")
imp_default, topcols_default = run_RFC(X_default, y_default,df_default_scaled)
default_fi = sns.barplot(x=imp_default.index[:20], y=imp_default['Feature Importance'][:20])
default_fi.set_xticklabels(default_fi.get_xticklabels(), rotation=90)

In [ ]:
pca = PCA(random_state=5)
pca.fit(X_default)
x_pca = pca.transform(X_default)
df_comp = pd.DataFrame(pca.components_[:20], columns=df_default_scaled.columns[:-1])
plt.figure(figsize=(12,6))
sns.heatmap(df_comp,cmap='plasma',)

In [ ]:
pca = PCA(random_state=5)
pca.fit(X_wine)
x_pca = pca.transform(X_wine)
df_comp = pd.DataFrame(pca.components_, columns=df_wine_scaled.columns[:-1])
plt.figure(figsize=(12,6))
sns.heatmap(df_comp,cmap='plasma',)

In [ ]:
imp_wine, topcols_wine = run_RFC(X_wine, y_wine, df_wine_scaled)
pca_wine = PCA(n_components=5,random_state=5).fit_transform(X_wine)
ica_wine = ICA(n_components=5,random_state=5).fit_transform(X_wine)
ica_wine /= ica_wine.std(axis=0)
rca_wine = RCA(n_components=5,random_state=5).fit_transform(X_wine)
rfc_wine = df_wine_scaled[topcols_wine]
rfc_wine = np.array(rfc_wine.values)

In [ ]:
ss_wine_pca = run_kmeans(pca_wine,y_wine,'PCA Phishing Data')
ss_wine_ica = run_kmeans(ica_wine,y_wine,'ICA Phishing Data')
ss_wine_rca = run_kmeans(rca_wine,y_wine,'RCA Phishing Data')
ss_wine_rfc = run_kmeans(rfc_wine,y_wine,'RFC Phishing Data')

In [ ]:
def squared_distance_comparison(original=None, transformed=None, transformation='pca', aic=None, bic=None, method = 'kmeans'):
    
    if method == 'kmeans':
        kclusters = list(np.arange(2,50,2))

        #Plot squared distances
        fig = plt.figure()
        ax = fig.add_subplot(111)
        ax.plot(kclusters, original, 'bx-', label='original')
        ax.plot(kclusters, transformed, label=transformation)
        plt.grid(True)
        plt.legend(loc='best')
        plt.xlabel('No. Clusters')
        plt.ylabel('Squared Distance')
        plt.title('Squared Distances from cluster centers')
        plt.show()
    
    else:
        kdist = list(np.arange(2,100,5))
        #Plot squared distances
        fig = plt.figure()
        ax = fig.add_subplot(111)
        ax.plot(kdist, aic[0], 'bx-', label='original aic')
        ax.plot(kdist, bic[0], 'rx-', label='original bic')
        ax.plot(kdist, aic[1], 'bo-',label= transformation + ' aic')
        ax.plot(kdist, bic[1], 'ro-',label= transformation + ' bic')
        plt.grid(True)
        plt.legend(loc='best')
        plt.xlabel('No. Distribution')
        plt.title('Squared Distances from cluster centers')
        plt.show()
        
        
    

In [ ]:
squared_distance_comparison(wine_squared_distances, ss_wine_pca, 'pca')
squared_distance_comparison(wine_squared_distances, ss_wine_ica, 'ica')
squared_distance_comparison(wine_squared_distances, ss_wine_rca, 'rca')
squared_distance_comparison(wine_squared_distances, ss_wine_rfc, 'rfc')

In [ ]:
km = KMeans(n_clusters=5)
km.fit(pca_wine)
preds = km.predict(pca_wine)
plot_cluster(km, pca_wine, y_wine, preds)

In [ ]:
km = KMeans(n_clusters=5)
km.fit(ica_wine)
preds = km.predict(ica_wine)
plot_cluster(km, ica_wine, y_wine, preds)

In [ ]:
km = KMeans(n_clusters=5)
km.fit(rca_wine)
preds = km.predict(rca_wine)
plot_cluster(km, rca_wine, y_wine, preds)

In [ ]:
km = KMeans(n_clusters=5)
km.fit(rfc_wine)
preds = km.predict(rfc_wine)
plot_cluster(km, rfc_wine, y_wine, preds)

In [ ]:
aic_wine_pca, bic_wine_pca, log_like_wine_pca = run_EM(pca_wine,y_wine,'PCA Phishing Data')
aic_wine_ica, bic_wine_ica, log_like_wine_ica  = run_EM(ica_wine,y_wine,'ICA Phishing Data')
aic_wine_rca, bic_wine_rca, log_like_wine_rca  = run_EM(rca_wine,y_wine,'RCA Phishing Data')
aic_wine_rfc, bic_wine_rfc, log_like_wine_rfc  = run_EM(rfc_wine,y_wine,'RFC Phishing Data')

In [ ]:
squared_distance_comparison(aic=[wine_aic, aic_wine_pca], bic=[wine_bic, bic_wine_pca], transformation='pca', method='em')
squared_distance_comparison(aic=[wine_aic, aic_wine_ica], bic=[wine_bic, bic_wine_ica], transformation='ica', method='em')
squared_distance_comparison(aic=[wine_aic, aic_wine_rca], bic=[wine_bic, bic_wine_rca], transformation='rca', method='em')
squared_distance_comparison(aic=[wine_aic, aic_wine_rfc], bic=[wine_bic, bic_wine_rfc], transformation='rfc', method='em')

In [ ]:
km = EM(n_components = 8)
km.fit(pca_wine)
preds = km.predict(pca_wine)
plt.scatter(pca_wine[:,0], pca_wine[:,1], c=preds, cmap='rainbow')

In [ ]:
km = EM(n_components=8)
km.fit(ica_wine)
preds = km.predict(ica_wine)
plt.scatter(ica_wine[:,0], ica_wine[:,1], c=preds, cmap='rainbow')

In [ ]:
km = EM(n_components=8)
km.fit(rca_wine)
preds = km.predict(rca_wine)
plt.scatter(rca_wine[:,0], rca_wine[:,1], c=preds, cmap='rainbow')

In [ ]:
km = EM(n_components=8)
km.fit(rfc_wine)
preds = km.predict(rfc_wine)
plt.scatter(rfc_wine[:,0], rfc_wine[:,1], c=preds, cmap='rainbow')

In [ ]:
imp_default, topcols_default = run_RFC(X_default, y_default, df_default_scaled)
pca_default = PCA(n_components = 15,random_state=5).fit_transform(X_default)
ica_default = ICA(n_components = 25,random_state=5).fit_transform(X_default)
rca_default = ICA(n_components = 20,random_state=5).fit_transform(X_default)
rfc_default = df_default_scaled[topcols_default]
rfc_default = np.array(rfc_default.values)

In [ ]:
ss_default_pca = run_kmeans(pca_default,y_default,'PCA Default Data')
ss_default_ica = run_kmeans(ica_default,y_default,'ICA Default Data')
ss_default_rca = run_kmeans(rca_default,y_default,'RCA Default Data')
ss_default_rfc = run_kmeans(rfc_default,y_default,'RFC Default Data')

In [ ]:
squared_distance_comparison(default_squared_distances, ss_default_pca, 'pca')
squared_distance_comparison(default_squared_distances, ss_default_ica, 'ica')
squared_distance_comparison(default_squared_distances, ss_default_rca, 'rca')
squared_distance_comparison(default_squared_distances, ss_default_rfc, 'rfc')

In [ ]:
aic_default_pca, bic_default_pca, log_like_default_pca = run_EM(pca_default,y_default,'PCA default Data')
aic_default_ica, bic_default_ica, log_like_default_ica  = run_EM(ica_default,y_default,'ICA default Data')
aic_default_rca, bic_default_rca, log_like_default_rca  = run_EM(rca_default,y_default,'RCA default Data')
aic_default_rfc, bic_default_rfc, log_like_default_rfc  = run_EM(rfc_default,y_default,'RFC default Data')

In [ ]:
squared_distance_comparison(aic=[default_aic, aic_default_pca], bic=[default_bic, bic_default_pca], transformation='pca', method='em')
squared_distance_comparison(aic=[default_aic, aic_default_ica], bic=[default_bic, bic_default_ica], transformation='ica', method='em')
squared_distance_comparison(aic=[default_aic, aic_default_rca], bic=[default_bic, bic_default_rca], transformation='rca', method='em')
squared_distance_comparison(aic=[default_aic, aic_default_rfc], bic=[default_bic, bic_default_rfc], transformation='rfc', method='em')

NN on Projected Data

In [ ]:
# Original, full dataset
X_train, X_test, y_train, y_test = train_test_split(np.array(X_wine),np.array(y_wine), test_size=0.20)
full_est = MLPClassifier(hidden_layer_sizes=(50,), solver='adam', activation='logistic', learning_rate_init=0.01, random_state=100)
train_samp_full, NN_train_score_full, NN_fit_time_full, NN_pred_time_full = plot_learning_curve(full_est, X_train, y_train,title="Neural Net Phishing: Full")
final_classifier_evaluation(full_est, X_train, X_test, y_train, y_test)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(np.array(pca_wine),np.array(y_wine), test_size=0.20)
pca_est = MLPClassifier(hidden_layer_sizes=(50,), solver='adam', activation='logistic', learning_rate_init=0.01, random_state=100)
train_samp_pca, NN_train_score_pca, NN_fit_time_pca, NN_pred_time_pca = plot_learning_curve(pca_est, X_train, y_train,title="Neural Net Phishing: PCA")
final_classifier_evaluation(pca_est, X_train, X_test, y_train, y_test)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(np.array(ica_wine),np.array(y_wine), test_size=0.20)
ica_est = MLPClassifier(hidden_layer_sizes=(50,), solver='adam', activation='logistic', learning_rate_init=0.01, random_state=100)
train_samp_ica, NN_train_score_ica, NN_fit_time_ica, NN_pred_time_ica = plot_learning_curve(ica_est, X_train, y_train,title="Neural Net Phishing: ICA")
final_classifier_evaluation(ica_est, X_train, X_test, y_train, y_test)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(np.array(rca_wine),np.array(y_wine), test_size=0.20)
rca_est = MLPClassifier(hidden_layer_sizes=(50,), solver='adam', activation='logistic', learning_rate_init=0.01, random_state=100)
train_samp_rca, NN_train_score_rca, NN_fit_time_rca, NN_pred_time_rca = plot_learning_curve(rca_est, X_train, y_train,title="Neural Net Phishing: RCA")
final_classifier_evaluation(rca_est, X_train, X_test, y_train, y_test)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(np.array(rfc_wine),np.array(y_wine), test_size=0.20)
rfc_est = MLPClassifier(hidden_layer_sizes=(50,), solver='adam', activation='logistic', learning_rate_init=0.01, random_state=100)
train_samp_rfc, NN_train_score_rfc, NN_fit_time_rfc, NN_pred_time_rfc = plot_learning_curve(rfc_est, X_train, y_train,title="Neural Net Phishing: RFC")
final_classifier_evaluation(rfc_est, X_train, X_test, y_train, y_test)

In [ ]:
def compare_fit_time(n,full_fit,pca_fit,ica_fit,rca_fit,rfc_fit,title):
    
    plt.figure()
    plt.title("Model Training Times: " + title)
    plt.xlabel("Training Examples")
    plt.ylabel("Model Training Time (s)")
    plt.plot(n, full_fit, '-', color="k", label="Full Dataset")
    plt.plot(n, pca_fit, '-', color="b", label="PCA")
    plt.plot(n, ica_fit, '-', color="r", label="ICA")
    plt.plot(n, rca_fit, '-', color="g", label="RCA")
    plt.plot(n, rfc_fit, '-', color="m", label="RFC")
    plt.legend(loc="best")
    plt.show()
    
def compare_pred_time(n,full_pred, pca_pred, ica_pred, rca_pred, rfc_pred, title):
    
    plt.figure()
    plt.title("Model Prediction Times: " + title)
    plt.xlabel("Training Examples")
    plt.ylabel("Model Prediction Time (s)")
    plt.plot(n, full_pred, '-', color="k", label="Full Dataset")
    plt.plot(n, pca_pred, '-', color="b", label="PCA")
    plt.plot(n, ica_pred, '-', color="r", label="ICA")
    plt.plot(n, rca_pred, '-', color="g", label="RCA")
    plt.plot(n, rfc_pred, '-', color="m", label="RFC")
    plt.legend(loc="best")
    plt.show()


def compare_learn_time(n,full_learn, pca_learn, ica_learn, rca_learn, rfc_learn, title):
    
    plt.figure()
    plt.title("Model Learning Rates: " + title)
    plt.xlabel("Training Examples")
    plt.ylabel("Model F1 Score")
    plt.plot(n, full_learn, '-', color="k", label="Full Dataset")
    plt.plot(n, pca_learn, '-', color="b", label="PCA")
    plt.plot(n, ica_learn, '-', color="r", label="ICA")
    plt.plot(n, rca_learn, '-', color="g", label="RCA")
    plt.plot(n, rfc_learn, '-', color="m", label="RFC")
    plt.legend(loc="best")
    plt.show() 

In [ ]:
compare_fit_time(train_samp_full, NN_fit_time_full, NN_fit_time_pca, NN_fit_time_ica, 
                 NN_fit_time_rca, NN_fit_time_rfc, 'Wine Dataset')              
compare_pred_time(train_samp_full, NN_pred_time_full, NN_pred_time_pca, NN_pred_time_ica, 
                 NN_pred_time_rca, NN_pred_time_rfc, 'Wine Dataset')   
compare_learn_time(train_samp_full, NN_train_score_full, NN_train_score_pca, NN_train_score_ica, 
                 NN_train_score_rca, NN_train_score_rfc, 'Wine Dataset')  

Projected Data with cluster labels on wine data

In [ ]:
def addclusters(X,km_lables,em_lables):
    
    df = pd.DataFrame(X)
    df['KM Cluster'] = km_labels
    df['EM Cluster'] = em_labels
    #col_1hot = ['KM Cluster', 'EM Cluster']
    #df_1hot = df[col_1hot]
    #df_1hot = pd.get_dummies(df_1hot).astype('category')
    #df_others = df.drop(col_1hot,axis=1)
    #df = pd.concat([df_others,df_1hot],axis=1)
    new_X = np.array(df.values)   
    
    return new_X

In [ ]:
km = KMeans(n_clusters=5,n_init=10,random_state=100,n_jobs=-1).fit(X_wine)
km_labels = km.labels_
em = EM(n_components=8,covariance_type='diag',n_init=1,warm_start=True,random_state=100).fit(X_wine)
em_labels = em.predict(X_wine)

clust_full = addclusters(X_wine,km_labels,em_labels)
clust_pca = addclusters(pca_wine,km_labels,em_labels)
clust_ica = addclusters(ica_wine,km_labels,em_labels)
clust_rca = addclusters(rca_wine,km_labels,em_labels)
clust_rfc = addclusters(rfc_wine,km_labels,em_labels)

In [ ]:
# Original, full dataset
X_train, X_test, y_train, y_test = train_test_split(np.array(clust_full),np.array(y_wine), test_size=0.20)
full_est = MLPClassifier(hidden_layer_sizes=(50,), solver='adam', activation='logistic', learning_rate_init=0.01, random_state=100)
train_samp_full, NN_train_score_full, NN_fit_time_full, NN_pred_time_full = plot_learning_curve(full_est, X_train, y_train,title="Neural Net Phishing with Clusters: Full")
final_classifier_evaluation(full_est, X_train, X_test, y_train, y_test)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(np.array(clust_pca),np.array(y_wine), test_size=0.20)
pca_est = MLPClassifier(hidden_layer_sizes=(50,), solver='adam', activation='logistic', learning_rate_init=0.01, random_state=100)
train_samp_pca, NN_train_score_pca, NN_fit_time_pca, NN_pred_time_pca = plot_learning_curve(pca_est, X_train, y_train,title="Neural Net Wine with Clusters: PCA")
final_classifier_evaluation(pca_est, X_train, X_test, y_train, y_test)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(np.array(clust_ica),np.array(y_wine), test_size=0.20)
ica_est = MLPClassifier(hidden_layer_sizes=(50,), solver='adam', activation='logistic', learning_rate_init=0.01, random_state=100)
train_samp_ica, NN_train_score_ica, NN_fit_time_ica, NN_pred_time_ica = plot_learning_curve(ica_est, X_train, y_train,title="Neural Net Wine with Clusters: ICA")
final_classifier_evaluation(ica_est, X_train, X_test, y_train, y_test)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(np.array(clust_rca),np.array(y_wine), test_size=0.20)
rca_est = MLPClassifier(hidden_layer_sizes=(50,), solver='adam', activation='logistic', learning_rate_init=0.01, random_state=100)
train_samp_rca, NN_train_score_rca, NN_fit_time_rca, NN_pred_time_rca = plot_learning_curve(rca_est, X_train, y_train,title="Neural Net Wine with Clusters: RCA")
final_classifier_evaluation(rca_est, X_train, X_test, y_train, y_test)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(np.array(clust_rfc),np.array(y_wine), test_size=0.20)
rfc_est = MLPClassifier(hidden_layer_sizes=(50,), solver='adam', activation='logistic', learning_rate_init=0.01, random_state=100)
train_samp_rfc, NN_train_score_rfc, NN_fit_time_rfc, NN_pred_time_rfc = plot_learning_curve(rfc_est, X_train, y_train,title="Neural Net Wine with Clusters: RFC")
final_classifier_evaluation(rfc_est, X_train, X_test, y_train, y_test)

In [ ]:
compare_fit_time(train_samp_full, NN_fit_time_full, NN_fit_time_pca, NN_fit_time_ica, 
                 NN_fit_time_rca, NN_fit_time_rfc, 'Wine Dataset')              
compare_pred_time(train_samp_full, NN_pred_time_full, NN_pred_time_pca, NN_pred_time_ica, 
                 NN_pred_time_rca, NN_pred_time_rfc, 'Wine Dataset')   
compare_learn_time(train_samp_full, NN_train_score_full, NN_train_score_pca, NN_train_score_ica, 
                 NN_train_score_rca, NN_train_score_rfc, 'Wine Dataset')  